## 做A股量化策略可视

量化策略是一个很有意思的方向，且不论是否赚钱，用数据来拟合趋势一直都让我心生向往。
鄙人做这个，一方面是学习量化交易的实作，另一方面是为尝试撰写其他的策略做准备。
tushare是个不错的包，只是需要贡献money or data来获得经验，毕竟也不是做慈善的，想来可以理解。
之后会考虑将逐步建立自己的金融数据库，方便各种意义上的使用。

本程序是基于以下链接的内容进行的改写，该作者贡献了很多量化程序的指导，十分感谢。
https://mp.weixin.qq.com/s?__biz=MzU3MDY4MjI2NA==&mid=2247483816&idx=1&sn=6266f603251f09fdbb78fba27aed517a&chksm=fceaff5fcb9d7649f5cc886833e739ac98a42299a40c03f14225d2300622edbd4aa9b7adb368&token=646845752&lang=zh_CN&scene=21#wechat_redirect

In [2]:
# 此处请各位去tushare自行注册获取token
import os
envX = os.environ
try:
    # 为了进一步保护token，我选择将token存在电脑的变量中
    tushare_token = envX['TUSHARE_TOKEN']
    print("token获得成功")
except:
    # 当然也可以选择手动输入
    tushare_token = input("请输入你的tushare密钥")

token获得成功


### 导入tushare包，并初始化

In [3]:
import tushare as ts

pro = ts.pro_api(tushare_token)

### 获取行情数据

In [40]:
# 日线行情从daily获得
# daily = pro.daily()
# daily

### 获取财务数据

In [41]:
# 利润表数据从income获得
# daily = pro.income()
# daily

## 量化策略
##### 假定其实金额100万元人民币，每个交易日以开盘价买入上一交易日成交额最大的股票，三天后卖出。买入数量为不超过且尽可能逼近当前总资产的1/time（time用于分隔portfolio的资金量），总资产的计算不考虑浮动盈亏。回测期间为2022年1月1日-2022年3月1日。

In [42]:
# 启动金额100万
start_amount = 1000000

### 每个交易日以开盘价买入上一交易日成交额最大的股票，三天后卖出。买入数量为不超过且尽可能逼近当前总资产的1/4，总资产的计算不考虑浮动盈亏。回测期间为2022年1月1日-2022年7月1日。

In [43]:
trade_date = pro.trade_cal(start_date=20220101, end_date=20220301, is_open=1)
trade_date

,exchange,cal_date,is_open,pretrade_date
0,SSE,20220104,1,20211231
1,SSE,20220105,1,20220104
2,SSE,20220106,1,20220105
3,SSE,20220107,1,20220106
4,SSE,20220110,1,20220107
5,SSE,20220111,1,20220110
6,SSE,20220112,1,20220111
7,SSE,20220113,1,20220112
8,SSE,20220114,1,20220113
9,SSE,20220117,1,20220114


In [44]:
# 回测第一天不发生交易，为了方便，我们把它计为第0天。交易日期如下
current_date = trade_date.loc[0, 'cal_date']
current_date

'20220104'

In [45]:
# 总资产为初始总资产，货币资金为总资产
amount = {}    # 总资产
currency_amount = {}      # 货币资产
amount[current_date] = currency_amount[current_date] = start_amount

In [46]:
import pandas as pd
trade_data = pd.DataFrame(columns=['ts_code', 'buy_date', 'buy_price', 'buy_num', 'hold_days', 'sale_date', 'sale_price'])
trade_data

,ts_code,buy_date,buy_price,buy_num,hold_days,sale_date,sale_price


### 开始交易

In [47]:
# 第一天
# 记录上个交易日
pre_date = current_date
# 获得当前交易日
current_date = trade_date.loc[1, 'cal_date']
# 获取上个交易日的成交量信息
pre_daily = pro.daily(trade_date = pre_date, fields='ts_code, amount')
# 获取上一交易日成交量最大的股票。
ts_code = pre_daily[pre_daily['amount']==pre_daily.amount.max()].iloc[0, 0]
# 获取本日开盘价
buy_price = pro.daily(trade_date = current_date, ts_code = ts_code).iloc[0, 2]
# 计算买入数量
buy_amount_limit = min(amount[pre_date] / 4, currency_amount[pre_date])
buy_num = buy_amount_limit // (buy_price * 100)
print("今天是{},昨日成交量最大股票为{}，其今日开盘价为{}，准备买入{}股。".format(current_date,ts_code,buy_price,buy_num*100))

今天是20220105,昨日成交量最大股票为300750.SZ，其今日开盘价为566.4，准备买入400.0股。


In [48]:
# 写入交易数据
trade_data.loc[len(trade_data),['ts_code', 'buy_date', 'buy_price', 'buy_num', 'hold_days', 'sale_price']] = [ts_code, current_date, buy_price, buy_num, 0, 0]
# 改写货币资金与总资产
currency_amount[current_date] = currency_amount[pre_date] - buy_price * buy_num * 100
amount[current_date] = amount[pre_date]
currency_amount

{'20220104': 1000000, '20220105': 773440.0}

### 第二天开始

In [49]:
time = 7
# 最后一天
# last_day = trade_date.loc[trade_date['cal_date'].last_valid_index(),'cal_date']
# 从第二天到最后一天
# while(current_date!=last_day):
for current_date in trade_date.loc[1:, 'cal_date']:
    # 记录交易日，转记上一日
    
    #current_date = trade_date.loc[days, 'cal_date']
    #days = days + 1
    
    # 有持有量多余3天的可作为交易对象
    trade_index = ((trade_data['hold_days']==time) & (trade_data['sale_price']==0))
    # trade_data[(trade_data.hold_days == 3) & (trade_data.sale_price == 0)]
    if(len(trade_index) != 0):
        # 设定卖出对象
        ts_code = trade_data.loc[len(trade_index)-1, 'ts_code']
        # 设定卖出数量
        sale_num = trade_data.loc[len(trade_index)-1, 'buy_num']
        orignal_buy_price = trade_data.loc[len(trade_index)-1, 'buy_price']
        # 今日结束卖价
        sale_price = pro.daily(trade_date = current_date, ts_code = ts_code).loc[0, 'close']
        # 写入交易数据
        trade_data.loc[trade_index, ['sale_date', 'sale_price']] = [current_date, sale_price]
        print("{}卖出{} {}股，盈亏{}".format(current_date,ts_code,sale_num*100,(sale_price - orignal_buy_price)*sale_num*100))
        
        
    # 之前买入的股票持有天数+1 
    trade_data.loc[trade_data['hold_days']<time, 'hold_days'] += 1
    pre_daily = pro.daily(trade_date = pre_date, fields='ts_code, amount')
    ts_code = pre_daily[pre_daily['amount']==pre_daily.amount.max()].iloc[0, 0]
    buy_price = pro.daily(trade_date = current_date, ts_code = ts_code).iloc[0, 2]
    buy_amount_limit = min(amount[pre_date] / (time+1), currency_amount[pre_date])
    buy_num = buy_amount_limit // (buy_price * 100)
    trade_data.loc[len(trade_data),['ts_code', 'buy_date', 'buy_price', 'buy_num', 'hold_days', 'sale_price']] = [ts_code, current_date, buy_price, buy_num, 0, 0]
    if(len(trade_index) != 0):
        # 后面有卖出的时日
        currency_amount[current_date] = currency_amount[pre_date] - buy_price * buy_num * 100 + sale_num * sale_price * 100
        amount[current_date] = amount[pre_date] + (sale_price - orignal_buy_price) * sale_num * 100
    else:
        # 前三天
        currency_amount[current_date] = currency_amount[pre_date] - buy_price * buy_num * 100
        amount[current_date] = amount[pre_date]
    print("余额{}".format(amount[current_date]))
    pre_date = current_date

20220105卖出300750.SZ 400.0股，盈亏-7559.999999999991
余额992440.0
20220106卖出300750.SZ 200.0股，盈亏-4985.99999999999
余额987454.0
20220107卖出600941.SH 2100.0股，盈亏-546.0000000000107
余额986908.0
20220110卖出600519.SH 0.0股，盈亏-0.0
余额986908.0
20220111卖出600031.SH 4800.0股，盈亏-3263.9999999999986
余额983644.0
20220112卖出300750.SZ 200.0股，盈亏4785.99999999999
余额988430.0
20220113卖出002594.SZ 400.0股，盈亏3724.000000000001
余额992154.0
20220114卖出300750.SZ 200.0股，盈亏1279.9999999999955
余额993434.0
20220117卖出600519.SH 0.0股，盈亏-0.0
余额993434.0
20220118卖出600111.SH 2800.0股，盈亏-3472.0000000000055
余额989962.0
20220119卖出300750.SZ 200.0股，盈亏-7372.000000000003
余额982590.0
20220120卖出002432.SZ 1800.0股，盈亏19043.999999999996
余额1001634.0
20220121卖出300750.SZ 200.0股，盈亏698.0000000000018
余额1002332.0
20220124卖出601318.SH 2300.0股，盈亏-850.9999999999941
余额1001481.0
20220125卖出600196.SH 2500.0股，盈亏-10325.000000000005
余额991156.0
20220126卖出000661.SZ 700.0股，盈亏762.9999999999825
余额991919.0
20220127卖出002460.SZ 800.0股，盈亏-2960.0000000000136
余额988959.0
20220128卖出688223.SH 11

In [50]:
amount

{'20220104': 1000000,
 '20220105': 992440.0,
 '20220106': 987454.0,
 '20220107': 986908.0,
 '20220110': 986908.0,
 '20220111': 983644.0,
 '20220112': 988430.0,
 '20220113': 992154.0,
 '20220114': 993434.0,
 '20220117': 993434.0,
 '20220118': 989962.0,
 '20220119': 982590.0,
 '20220120': 1001634.0,
 '20220121': 1002332.0,
 '20220124': 1001481.0,
 '20220125': 991156.0,
 '20220126': 991919.0,
 '20220127': 988959.0,
 '20220128': 989077.0,
 '20220207': 989077.0,
 '20220208': 976049.0,
 '20220209': 969601.0,
 '20220210': 965421.0,
 '20220211': 965421.0,
 '20220214': 965821.0,
 '20220215': 974021.0,
 '20220216': 976529.0,
 '20220217': 977929.0,
 '20220218': 977433.0,
 '20220221': 981580.0,
 '20220222': 977080.0,
 '20220223': 980135.0,
 '20220224': 980135.0,
 '20220225': 984295.0,
 '20220228': 984295.0,
 '20220301': 986731.0}

In [51]:
# 然而最终这个策略没能赚钱，很难赚钱啊看来
trade_data

,ts_code,buy_date,buy_price,buy_num,hold_days,sale_date,sale_price
0,300750.SZ,20220105,566.4,4.0,7,20220114,577.4
1,300750.SZ,20220105,566.4,2.0,7,20220117,1861.61
2,600941.SH,20220106,57.88,21.0,7,20220118,42.33
3,600519.SH,20220107,1975.0,0.0,7,20220119,569.99
4,600031.SH,20220110,25.2,48.0,7,20220120,77.66
5,300750.SZ,20220111,541.11,2.0,7,20220121,569.49
6,002594.SZ,20220112,246.0,4.0,7,20220124,53.13
7,300750.SZ,20220113,571.0,2.0,7,20220125,45.0
8,600519.SH,20220114,1877.39,0.0,7,20220126,168.17
9,600111.SH,20220117,43.57,28.0,7,20220127,136.2
